In [95]:
import pandas as pd
from sqlalchemy import create_engine
from konlpy.tag import Kkma
from konlpy.utils import pprint


In [132]:
q = '''
SELECT
   TRIM(ref_term) AS ref_term,
   content_id,
   COUNT(content_id) AS label 
FROM
   bun_log_app.app_event_type_view 
WHERE
   YEAR || MONTH || DAY || HOUR >= '2020020315' 
   AND YEAR || MONTH || DAY || HOUR <= '2020020318' 
   AND user_id != '-1' 
   AND content_id != '-1' 
   AND ref_term IS NOT NULL 
   AND ref_term != '...' 
   AND content_id IS NOT NULL 
GROUP BY
   TRIM(ref_term),
   content_id 
HAVING
   COUNT(content_id) >= 3'''
df = pd.read_sql(q, con=redshift)

In [133]:
kkma = Kkma()
nouns = [' '.join(kkma.nouns(i)) for i in df['ref_term'].tolist()]
sentences = [' '.join(kkma.sentences(i)) for i in df['ref_term'].tolist()]

In [140]:
df['nouns'] = nouns
df['sentences'] = sentences

In [217]:
df.head()

,ref_term,content_id,label,nouns,sentences
0,베이퍼맥스,116039729,3,베이 베이퍼맥스 퍼맥스,베이 퍼맥스
1,솔로이스트,110951649,4,솔로 솔로이스트 이스트,솔로 이스트
2,나이키,114052271,3,나이키,나이키
3,키르시 가디건,111168473,3,키 키르 르 디 디건 건,키 르시 가 디 건
4,2021,116531869,4,2021,2021


In [218]:
ref_term = df['sentences'].unique()
row_number = range(len(ref_term))
ref_term_label = pd.DataFrame.from_dict({'ref_term': ref_term, 'label': row_number})
ref_term_label = ref_term_label.sort_values('ref_term')
ref_term_label['ref_term'] = ref_term_label['ref_term'].astype(str)
ref_term_label.to_csv('./ref_term_label.csv', index=False, header=False)

In [219]:
content_id = df['content_id'].unique()
row_number = range(len(content_id))
content_id_label = pd.DataFrame.from_dict({'content_id': content_id, 'label': row_number})
content_id_label = content_id_label.sort_values('content_id')
content_id_label['content_id'] = content_id_label['content_id'].astype(str)
content_id_label.to_csv('./content_id_label.csv', index=False, header=False)

In [220]:
x = df[['ref_term', 'content_id', 'label']]
y1 = content_id_label
y2 = ref_term_label
keyword_label = pd.merge(x, y1, on='content_id')
keyword_label = pd.merge(keyword_label, y2, on='ref_term')
keyword_label.columns = ['ref_term', 'content_id', 'label', 'content_id_label', 'ref_term_label']
keyword_label['max_content_id_label'] = keyword_label['content_id_label'].max()
keyword_label['max_ref_term_label'] = keyword_label['ref_term_label'].max()

In [221]:
for column in keyword_label.columns:
    keyword_label[column] = keyword_label[column].astype(str)

In [222]:
libsvm = []
for i in range(len(keyword_label)):
    r = keyword_label.iloc[i]
    s = r['label'] + ' ' + r['content_id_label'] + ':1 ' + str(int(r['ref_term_label']) + int(r['max_content_id_label'])) + ':1'
    libsvm.append(s)
libsvm_csv = pd.DataFrame.from_dict({'libsvm': libsvm})
libsvm_csv.to_csv('./libsvm.csv', index=False, header=False)

In [101]:
df['new_keyword'] = new_keyword
group = df.groupby('new_keyword', as_index=False)['label'].sum()

In [130]:

df.to_csv('./konlpy.csv')

In [117]:
l = []
for i in df['ref_term'].tolist():
    words = kkma.nouns(i)
    for w in words:
        if w not in l:
            l.append(w)

In [129]:
kkma.nouns('갤럭시10')

['럭', '럭시10', '시', '10']